# Comparison of spike shapes, amplitudes, and detectable volume for different cell types

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
import numpy as np
import LFPy
import brainsignals.neural_simulations as ns
from brainsignals.plotting_convention import mark_subplots

np.random.seed(12345)
ns.load_mechs_from_folder(ns.cell_models_folder)

cell_title_dict = {"L5_MC_bAC217_1": "L5 martinotti cell",
                  "L5_TTPC2_cADpyr232_2": "L5 pyramidal cell",
                  "L5_NGC_bNAC219_5": "L5 neurogliaform cell"}

# sim duration
tstop = 150.
dt = 2**-5

In [ ]:
def insert_current_stimuli(cell, amp=-0.1):
    stim_params = {'amp': amp,
                   'idx': 0,
                   'pptype': "ISyn",
                   'dur': 1e9,
                   'delay': 0}

    synapse = LFPy.StimIntElectrode(cell, **stim_params)
    return synapse, cell

# Grid around cells to investigate spike in and plot
xmin, xmax = [-150, 150]
zmin, zmax = [-150, 150]

dx = 5.
dz = 5.
x_grid, z_grid = np.mgrid[xmin:xmax+dx:dx, zmin:zmax+dz:dz]
num_elecs = len(x_grid.flatten())
elec_grid_params = dict(
            sigma = 0.3,      # extracellular conductivity
            x = x_grid.flatten(),
            y = np.zeros(num_elecs),
            z = z_grid.flatten(),
            method = 'root_as_point',
        )
soma_elec_idx = np.argmin((elec_grid_params["x"] - 10)**2 + 
                          (elec_grid_params["y"] - 0)**2 + 
                          (elec_grid_params["z"] - 0)**2)

## We will just delete everything except a small window around a spike

In [ ]:
def pick_out_spike(cell):
    # spike sampling
    threshold = -20  # spike threshold (mV)
    samplelength = int(4. / dt)

    # detect action potentials from intracellular trace
    crossings = ((cell.somav[:-1] < threshold) &
                 (cell.somav[1:] >= threshold))
    spike_inds = np.where(crossings)[0]
    if len(spike_inds) == 0:
        print("No spike, need stronger amplitude input!")
    last_spike_t_idx = spike_inds[-1]
    # In case last spike is cut short by end of simulation:
    if last_spike_t_idx + samplelength > len(cell.somav):
        last_spike_t_idx = spike_inds[-2]

    t0_idx = last_spike_t_idx - samplelength
    t1_idx = last_spike_t_idx + samplelength
    cell.imem = cell.imem[:, t0_idx:t1_idx]
    cell.somav = cell.somav[t0_idx:t1_idx]
    cell.tvec = cell.tvec[t0_idx:t1_idx] - cell.tvec[t0_idx]
    return cell

## We define a plotting function

In [ ]:
def plot_cell(cell, cell_name, elec, eaps, neuron_idx):
    fig = plt.figure(figsize=(2, 2.33))
    fig.subplots_adjust(left=0, right=0.96, hspace=0.4, wspace=0.2)

    zips = []
    for x, z in cell.get_idx_polygons(projection=('x', 'z')):
        zips.append(list(zip(x, z)))
    polycol = PolyCollection(zips,
                             edgecolors='none',
                             facecolors='gray')
    ax_morph = fig.add_axes([-0.01, 0.3, 0.76, 0.6], aspect=1, 
                            title=cell_title_dict[cell_name],
                               #xlabel='µm', ylabel='µm',
                               xlim=[xmin, xmax], ylim=[zmin, zmax],
                               frameon=False,
                               xticks=[], yticks=[])
    ax_morph.add_collection(polycol)
    cax = fig.add_axes([0.73, 0.5, 0.015, 0.35])
    ax_morph.plot(elec.x[soma_elec_idx], elec.z[soma_elec_idx], 'r.')
    eaps_p2p = np.max(eaps, axis=1) - np.min(eaps, axis=1)
    
    ax_morph.plot([-100, -80], [-100, -100], lw=1, c='k')
    ax_morph.text(-90, -110, "20 µm", ha="center", va="top")
    
    detection_threshold = 30.
    
    levels_norm = [0, detection_threshold, 1e9]#scale_max * levels
    colors_from_map = ['0.95', '#ffbbbb', (0.5, 0.5, 0.5, 1)]
    ep_intervals = ax_morph.contourf(x_grid, z_grid, 
                                     eaps_p2p.reshape(x_grid.shape),
                                     zorder=-2, colors=colors_from_map,
                                     levels=levels_norm, extend='both')
    
    cbar = plt.colorbar(ep_intervals, cax=cax)
    
    cbar.set_ticks(np.array([detection_threshold / 2, 1e9/2]))
    cbar.set_ticklabels(np.array(["<%d µV" % detection_threshold, 
                                  ">%d µV" % detection_threshold]))
    ax_vmem = fig.add_axes([0.01, 0.05, 0.38, 0.17], frameon=False, ylim=[-80, 30],
                           xticks=[], yticks=[], title=r"soma $V_{\rm m}$")
    ax_vmem.plot(cell.tvec, cell.somav, 'k')
    ax_vmem.plot([cell.tvec[-1] - 1.9, cell.tvec[-1] - 1.9], [-50, 0], lw=1, c='k')
    ax_vmem.text(cell.tvec[-1] - 1.5, -35, "50 mV")
    
    ax_vmem.plot([3.5, 4.5], [cell.somav[0] - 5, cell.somav[0] - 5], lw=1, c='k')
    ax_vmem.text(3.75, cell.somav[0] - 10, "1 ms", va="top", ha="center")
    ax_eap = fig.add_axes([0.4, 0.05, 0.38, 0.17], frameon=False, 
                           xticks=[], yticks=[], title="spike")
    ax_eap.plot(cell.tvec, eaps[soma_elec_idx], c='r')
    ax_eap.plot([3.5, 4.5], [np.min(eaps[soma_elec_idx]) * 1.1, 
                             np.min(eaps[soma_elec_idx]) * 1.1], lw=1, c='k')
    ax_eap.text(3.75, np.min(eaps[soma_elec_idx]) * 1.15, "1 ms", va="top", ha="center")
    
    ax_eap.plot([cell.tvec[-1] - 2.5, cell.tvec[-1] - 2.5], 
                [np.min(eaps[soma_elec_idx]), 0], lw=1, c='k')
    ax_eap.text(cell.tvec[-1] - 2.4, np.min(eaps[soma_elec_idx] / 2), 
                "{:1.0f} µV".format(-np.min(eaps[soma_elec_idx])), va="center")
    
    mark_subplots(ax_morph, "ABC"[neuron_idx], ypos=1.1, xpos=0.02)
    fig.savefig("eap_%s_10um.pdf" % cell_name)
    # return fig

In [ ]:
# Names of neurons from https://bbp.epfl.ch/nmc-portal/welcome.html
# Have chosen one pyramidal cell and two interneurons from L5
neurons = ["L5_TTPC2_cADpyr232_2",
           "L5_MC_bAC217_1",
           "L5_NGC_bNAC219_5",
           ]
# Cells are very different in size, and therefore we use different stimuli amplitude
stim_amps = [-0.3, -0.2, -0.1]

for neuron_idx in range(len(neurons)):
    cell_name = neurons[neuron_idx]
    stim_amp = stim_amps[neuron_idx]

    cell = ns.return_BBP_neuron(cell_name, tstop, dt)
    syn, cell = insert_current_stimuli(cell, stim_amp)
    cell.simulate(rec_imem=True)
    cell = pick_out_spike(cell)

    elec = LFPy.RecExtElectrode(cell, **elec_grid_params)
    M_elec = elec.get_transformation_matrix()
    eaps = M_elec @ cell.imem * 1000

    plot_cell(cell, cell_name, elec, eaps, neuron_idx)
    del cell
    del syn
del elec
    